# Quantum State Tomography

In [1]:
from pprint import pprint
import qiskit
import qiskit_experiments as qe
tomo = qe.tomography

# For simulation
from qiskit.providers.aer import AerSimulator
from qiskit.test.mock import FakeParis

# Noisy simulator backend
backend = AerSimulator.from_backend(FakeParis())

## State Tomography Experiment

To run a state tomography experiment we initialize the experiment with a circuit to prepare the state to be measured. We can also pass in an `Operator`, or a `Statevector` to describe the preparation circuit.

In [2]:
# Run experiments

# GHZ State preparation circuit
nq = 2
qc_ghz = qiskit.QuantumCircuit(nq)
qc_ghz.h(0)
qc_ghz.s(0)
for i in range(1, nq):
    qc_ghz.cx(0, i)

# QST Experiment
qstexp1 = tomo.StateTomography(qc_ghz)
qstdata1 = qstexp1.run(backend, seed_simulation=100)
qstresult = qstdata1.analysis_result(-1)

print('FIT RESULT')
pprint(qstresult)

FIT RESULT
{'fitter_metadata': {'fitter': 'linear_inversion',
                     'fitter_time': 0.0015842914581298828},
 'state': DensityMatrix([[ 0.47981771+0.j        ,  0.00439453-0.0086263j ,
                 0.00260417-0.00651042j, -0.01416016-0.44482422j],
               [ 0.00439453+0.0086263j ,  0.01920573+0.j        ,
                -0.00537109+0.00341797j,  0.01432292+0.00130208j],
               [ 0.00260417+0.00651042j, -0.00537109-0.00341797j,
                 0.03450521+0.j        ,  0.00146484-0.02034505j],
               [-0.01416016+0.44482422j,  0.01432292-0.00130208j,
                 0.00146484+0.02034505j,  0.46647135+0.j        ]],
              dims=(2, 2)),
 'state_fidelity': 0.91796875,
 'state_metadata': {'eigvals': array([0.91887418, 0.0471276 , 0.01804304, 0.01595518]),
                    'positive': True,
                    'positive_delta': 0.0,
                    'trace': 1.0000000000000002},
 'success': True}


### Tomography Results

The main results for tomography are the fitted state, which is stored in the `"state"` key as a `DensityMatrix` object:

In [3]:
print(qstresult["state"])

DensityMatrix([[ 0.47981771+0.j        ,  0.00439453-0.0086263j ,
                 0.00260417-0.00651042j, -0.01416016-0.44482422j],
               [ 0.00439453+0.0086263j ,  0.01920573+0.j        ,
                -0.00537109+0.00341797j,  0.01432292+0.00130208j],
               [ 0.00260417+0.00651042j, -0.00537109-0.00341797j,
                 0.03450521+0.j        ,  0.00146484-0.02034505j],
               [-0.01416016+0.44482422j,  0.01432292-0.00130208j,
                 0.00146484+0.02034505j,  0.46647135+0.j        ]],
              dims=(2, 2))


The state fidelity of the fitted state with the ideal state prepared by the input circuit is stored in the `"state_fidelity"` result field. Note that if the input circuit contained any measurements the ideal state cannot be automatically generated and this field will be set to `None`.

In [4]:
print("State Fidelity = {:.5f}".format(qstresult["state_fidelity"]))

State Fidelity = 0.91797


#### Additional state metadata

Additional data is stored in the tomography under the `"state_metadata"` field. This includes
- `eigvals`: the eigenvalues of the fitted state
- `trace`: the trace of the fitted state
- `positive`: Whether the eigenvalues are all non-negative
- `positive_delta`: the deviation from positivity given by 1-norm of negative eigenvalues.

If trace rescaling was performed this dictionary will also contain a `raw_trace` field containing the trace before rescaling.
Futhermore, if the state was rescaled to be positive or trace 1 an additional field `raw_eigvals` will contain the state eigenvalues before rescaling was performed.

In [5]:
pprint(qstresult["state_metadata"])

{'eigvals': array([0.91887418, 0.0471276 , 0.01804304, 0.01595518]),
 'positive': True,
 'positive_delta': 0.0,
 'trace': 1.0000000000000002}


To see the effect of rescaling we can perform a "bad" fit with very low counts

In [6]:
# QST Experiment
bad_data = qstexp1.run(backend, shots=10, seed_simulation=100)
bad_result = bad_data.analysis_result(-1)

print(bad_result["state_fidelity"])
pprint(bad_result["state_metadata"])

0.8405316844521707
{'eigvals': array([0.89168942, 0.10831058, 0.        , 0.        ]),
 'positive': True,
 'positive_delta': 0.0,
 'raw_eigvals': array([ 1.05462832,  0.27124948, -0.06026308, -0.26561472]),
 'trace': 0.9999999999999993}


## Tomography Fitters

The default fitters is `linear_inversion`, which reconstructs the state using *dual basis* of the tomography basis. This will typically result in a non-postive reconstructed state. This state is rescaled to be postive-semidfinite (PSD) by computing its eigen-decomposition and rescaling its eigenvalues using the approach from *J Smolin, JM Gambetta, G Smith, Phys. Rev. Lett. 108, 070502 (2012), [open access](https://arxiv.org/abs/arXiv:1106.5458).

There are several other fitters are included (See API documentation for details). For example if `cvxpy` is installed we can use the `cvxpy_gaussian_lstsq` fitter which allows constraining the fit to be PSD without requiring rescaling.

In [7]:
qstexp1.run_analysis(qstdata1, fitter='cvxpy_linear_lstsq')
qstresult2 = qstdata1.analysis_result(-1)

print('FIT RESULT')
pprint(qstresult2)

FIT RESULT
{'fitter_metadata': {'cvxpy_solver': 'CVXOPT',
                     'cvxpy_status': 'optimal',
                     'fitter': 'cvxpy_linear_lstsq',
                     'fitter_time': 0.057753801345825195},
 'state': DensityMatrix([[ 0.47981944+0.00000000e+00j,  0.00439546-8.62489044e-03j,
                 0.00260244-6.51111631e-03j, -0.01416025-4.44827007e-01j],
               [ 0.00439546+8.62489044e-03j,  0.01920331+0.00000000e+00j,
                -0.00537229+3.41858268e-03j,  0.01432449+1.30294300e-03j],
               [ 0.00260244+6.51111631e-03j, -0.00537229-3.41858268e-03j,
                 0.03450426-8.67361738e-19j,  0.00146412-2.03467865e-02j],
               [-0.01416025+4.44827007e-01j,  0.01432449-1.30294300e-03j,
                 0.00146412+2.03467865e-02j,  0.46647299+1.73472348e-18j]],
              dims=(2, 2)),
 'state_fidelity': 0.9179732199819044,
 'state_metadata': {'eigvals': array([0.91887865, 0.04713024, 0.01804001, 0.0159511 ]),
                    

## Parallel Tomography Experiment

We can also use the `qiskit_experiments.ParallelExperiment` class to run subsystem tomography on multiple qubits in parallel.

For example if we want to perform 1-qubit QST on several qubits at once:

In [8]:
from math import pi
num_qubits = 5
gates = [qiskit.circuit.library.RXGate(i * pi / (num_qubits - 1))
         for i in range(num_qubits)]

subexps = [
    tomo.StateTomography(gate, qubits=[i])
    for i, gate in enumerate(gates)
]
parexp = qe.composite.ParallelExperiment(subexps)
pardata = parexp.run(backend, seed_simulation=100)
print(pardata.analysis_result(-1))


- experiment_types: ['StateTomography', 'StateTomography', 'StateTomography', 'StateTomography', 'StateTomography']
- experiment_ids: ['e8025a8b-edcb-483d-b1a6-131ca638c7d2', 'f8f971b7-d99d-4d16-bfdf-c8d489b0d0f1', '15b3f86a-b34d-4f8d-9167-e70aa9efdcc9', '9f35847c-d601-41b3-a2e8-57a701023a16', 'cc2c46b9-292b-4017-b353-465271580b5e']
- experiment_qubits: [(0,), (1,), (2,), (3,), (4,)]
- success: True


View component experiment analysis results

In [9]:
for i in range(parexp.num_experiments):
    expdata = pardata.component_experiment_data(i)
    result = expdata.analysis_result(-1)
    
    print(f'\nPARALLEL EXP {i}: FIT RESULT')
    pprint(result)


PARALLEL EXP 0: FIT RESULT
{'fitter_metadata': {'fitter': 'linear_inversion',
                     'fitter_time': 0.00016832351684570312},
 'state': DensityMatrix([[ 0.98339844+0.j        , -0.01953125-0.01757812j],
               [-0.01953125+0.01757812j,  0.01660156+0.j        ]],
              dims=(2,)),
 'state_fidelity': 0.9833984374999999,
 'state_metadata': {'eigvals': array([0.98411208, 0.01588792]),
                    'positive': True,
                    'positive_delta': 0.0,
                    'trace': 1.0000000000000002},
 'success': True}

PARALLEL EXP 1: FIT RESULT
{'fitter_metadata': {'fitter': 'linear_inversion',
                     'fitter_time': 9.083747863769531e-05},
 'state': DensityMatrix([[0.8515625 +0.j        , 0.00878906+0.31933594j],
               [0.00878906-0.31933594j, 0.1484375 +0.j        ]],
              dims=(2,)),
 'state_fidelity': 0.9743968346437093,
 'state_metadata': {'eigvals': array([0.97502514, 0.02497486]),
                    'positiv